In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import KFold

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

pd.set_option('max_columns', 300)

kfolds = KFold(n_splits=5, shuffle=True, random_state=345)

In [2]:
df = pd.read_csv('data/processed_men/training_data.csv')

df.head()

,Season,DayNum,Team1,Team2,target,target_points,T1_Loc,T2_Loc,ID,T1_Ast,T1_Ast_advantage,T1_Ast_diff,T1_Away,T1_Blk,T1_Blk_advantage,T1_Blk_diff,T1_DR,T1_DR_advantage,T1_DR_diff,T1_DR_opportunity,T1_DR_opportunity_advantage,T1_DR_opportunity_diff,T1_FGA,T1_FGA2,T1_FGA2_advantage,T1_FGA2_diff,T1_FGA3,T1_FGA3_advantage,T1_FGA3_diff,T1_FGA_advantage,T1_FGA_diff,T1_FGM,T1_FGM2,T1_FGM2_advantage,T1_FGM2_diff,T1_FGM3,T1_FGM3_advantage,T1_FGM3_diff,T1_FGM_advantage,T1_FGM_diff,T1_FGM_no_ast,T1_FGM_no_ast_advantage,T1_FGM_no_ast_diff,T1_FTA,T1_FTA_advantage,T1_FTA_diff,T1_FTM,T1_FTM_advantage,T1_FTM_diff,T1_N_wins,T1_OR,T1_OR_advantage,T1_OR_diff,T1_OR_opportunity,T1_OR_opportunity_advantage,T1_OR_opportunity_diff,T1_OT_win,T1_PF,T1_PF_advantage,T1_PF_diff,T1_Score,T1_Score_advantage,T1_Score_diff,T1_Stl,T1_Stl_advantage,T1_Stl_diff,T1_TO,T1_TO_advantage,T1_TO_diff,T1_TO_perposs,T1_TO_perposs_advantage,T1_TO_perposs_diff,T1_Tot_Reb,T1_Tot_Reb_advantage,T1_Tot_Reb_diff,T1_True_shooting_perc_advantage,T1_True_shooting_perc_diff,T1_def_rating,T1_def_rating_advantage,T1_def_rating_diff,T1_impact,T1_impact_advantage,T1_impact_diff,T1_off_rating,T1_off_rating_advantage,T1_off_rating_diff,T1_opp_FGA,T1_opp_FTA,T1_opp_possessions,T1_opp_score,T1_opp_shtg_opportunity,T1_possessions,T1_possessions_advantage,T1_possessions_diff,T1_shtg_opportunity,T1_shtg_opportunity_advantage,T1_shtg_opportunity_diff,T1_top_team,T1_upset,T1_FGM_perc,T1_FGM2_perc,T1_FGM3_perc,T1_FT_perc,T1_FGM_no_ast_perc,T1_True_shooting_perc,T1_Opp_True_shooting_perc,T1_OT_win_perc,T1_Seed,T1_Rank,T2_Ast,T2_Ast_advantage,T2_Ast_diff,T2_Away,T2_Blk,T2_Blk_advantage,T2_Blk_diff,T2_DR,T2_DR_advantage,T2_DR_diff,T2_DR_opportunity,T2_DR_opportunity_advantage,T2_DR_opportunity_diff,T2_FGA,T2_FGA2,T2_FGA2_advantage,T2_FGA2_diff,T2_FGA3,T2_FGA3_advantage,T2_FGA3_diff,T2_FGA_advantage,T2_FGA_diff,T2_FGM,T2_FGM2,T2_FGM2_advantage,T2_FGM2_diff,T2_FGM3,T2_FGM3_advantage,T2_FGM3_diff,T2_FGM_advantage,T2_FGM_diff,T2_FGM_no_ast,T2_FGM_no_ast_advantage,T2_FGM_no_ast_diff,T2_FTA,T2_FTA_advantage,T2_FTA_diff,T2_FTM,T2_FTM_advantage,T2_FTM_diff,T2_N_wins,...,T2_TO_diff,T2_TO_perposs,T2_TO_perposs_advantage,T2_TO_perposs_diff,T2_Tot_Reb,T2_Tot_Reb_advantage,T2_Tot_Reb_diff,T2_True_shooting_perc_advantage,T2_True_shooting_perc_diff,T2_def_rating,T2_def_rating_advantage,T2_def_rating_diff,T2_impact,T2_impact_advantage,T2_impact_diff,T2_off_rating,T2_off_rating_advantage,T2_off_rating_diff,T2_opp_FGA,T2_opp_FTA,T2_opp_possessions,T2_opp_score,T2_opp_shtg_opportunity,T2_possessions,T2_possessions_advantage,T2_possessions_diff,T2_shtg_opportunity,T2_shtg_opportunity_advantage,T2_shtg_opportunity_diff,T2_top_team,T2_upset,T2_FGM_perc,T2_FGM2_perc,T2_FGM3_perc,T2_FT_perc,T2_FGM_no_ast_perc,T2_True_shooting_perc,T2_Opp_True_shooting_perc,T2_OT_win_perc,T2_Seed,T2_Rank,delta_Loc,delta_Ast,delta_Ast_advantage,delta_Ast_diff,delta_Away,delta_Blk,delta_Blk_advantage,delta_Blk_diff,delta_DR,delta_DR_advantage,delta_DR_diff,delta_DR_opportunity,delta_DR_opportunity_advantage,delta_DR_opportunity_diff,delta_FGA,delta_FGA2,delta_FGA2_advantage,delta_FGA2_diff,delta_FGA3,delta_FGA3_advantage,delta_FGA3_diff,delta_FGA_advantage,delta_FGA_diff,delta_FGM,delta_FGM2,delta_FGM2_advantage,delta_FGM2_diff,delta_FGM3,delta_FGM3_advantage,delta_FGM3_diff,delta_FGM_advantage,delta_FGM_diff,delta_FGM_no_ast,delta_FGM_no_ast_advantage,delta_FGM_no_ast_diff,delta_FTA,delta_FTA_advantage,delta_FTA_diff,delta_FTM,delta_FTM_advantage,delta_FTM_diff,delta_N_wins,delta_OR,delta_OR_advantage,delta_OR_diff,delta_OR_opportunity,delta_OR_opportunity_advantage,delta_OR_opportunity_diff,delta_OT_win,delta_PF,delta_PF_advantage,delta_PF_diff,delta_Score,delta_Score_advantage,delta_Score_diff,delta_Stl,delta_Stl_advantage,delta_Stl_diff,delta_TO,delta_TO_advantage,delta_TO_diff,delta_TO_perposs,delta_TO_perposs_advantage,delta_TO_perposs_diff,delta_Tot_Reb,delta_Tot_Reb_advantage,delta_Tot_Reb_diff,delta_True_shooting_perc_advantage,delta

In [3]:
ranks = pd.read_csv('data/raw_men/MDataFiles_Stage2/MMasseyOrdinals.csv')

ranks.head()

,Season,RankingDayNum,SystemName,TeamID,OrdinalRank
0,2003,35,SEL,1102,159
1,2003,35,SEL,1103,229
2,2003,35,SEL,1104,12
3,2003,35,SEL,1105,314
4,2003,35,SEL,1106,260


In [4]:
ranks.SystemName.nunique()

179

In [5]:
ranks.groupby(['Season', 'SystemName']).OrdinalRank.agg(['min', 'max', 'mean']).describe()

,min,max,mean
count,1013.0,1013.000000,1013.000000
mean,1.0,325.027641,162.856192
std,0.0,76.887084,38.459526
min,1.0,25.000000,12.978947
25%,1.0,341.000000,170.239316
50%,1.0,347.000000,174.000000
75%,1.0,351.000000,176.000000
max,1.0,353.000000,177.000000


In [6]:
tmp = ranks.groupby(['Season', 'SystemName']).OrdinalRank.agg(['min', 'max', 'mean']).reset_index()

tmp[tmp['max'] < 300].SystemName.unique()

array(['AP', 'USA', 'DES', 'LYN', 'ACU', 'TRX', 'D1A', 'JNG', 'BNT'],
      dtype=object)

In [7]:
ranks = ranks[~(ranks.SystemName.isin(['AP', 'USA', 'DES', 'LYN', 'ACU', 'TRX', 'D1A', 'JNG', 'BNT']))].copy()

ranks.head()

,Season,RankingDayNum,SystemName,TeamID,OrdinalRank
0,2003,35,SEL,1102,159
1,2003,35,SEL,1103,229
2,2003,35,SEL,1104,12
3,2003,35,SEL,1105,314
4,2003,35,SEL,1106,260


In [8]:
ranks.groupby(['Season', 'SystemName']).OrdinalRank.agg(['min', 'max', 'mean']).describe()

,min,max,mean
count,932.0,932.000000,932.000000
mean,1.0,345.386266,173.057840
std,0.0,8.046518,4.105360
min,1.0,324.000000,159.988095
25%,1.0,343.000000,171.675758
50%,1.0,347.000000,174.000000
75%,1.0,351.000000,176.000000
max,1.0,353.000000,177.000000


In [9]:
regular_season = 'data/raw_men/MDataFiles_Stage2/MRegularSeasonDetailedResults.csv'

reg = pd.read_csv(regular_season)

reg.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,WFGM3,WFGA3,WFTM,WFTA,WOR,WDR,WAst,WTO,WStl,WBlk,WPF,LFGM,LFGA,LFGM3,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2003,10,1104,68,1328,62,N,0,27,58,3,14,11,18,14,24,13,23,7,1,22,22,53,2,10,16,22,10,22,8,18,9,2,20
1,2003,10,1272,70,1393,63,N,0,26,62,8,20,10,19,15,28,16,13,4,4,18,24,67,6,24,9,20,20,25,7,12,8,6,16
2,2003,11,1266,73,1437,61,N,0,24,58,8,18,17,29,17,26,15,10,5,2,25,22,73,3,26,14,23,31,22,9,12,2,5,23
3,2003,11,1296,56,1457,50,N,0,18,38,3,9,17,31,6,19,11,12,14,2,18,18,49,6,22,8,15,17,20,9,19,4,3,23
4,2003,11,1400,77,1208,71,N,0,30,61,6,14,11,13,17,22,12,14,4,4,20,24,62,6,16,17,27,21,15,12,10,7,1,14


In [10]:
mean_ranks = ranks.groupby(['Season', 'TeamID', 'RankingDayNum'], as_index=False).OrdinalRank.mean()

mean_ranks.sample(10)

,Season,TeamID,RankingDayNum,OrdinalRank
120013,2010,1257,77,43.533333
187544,2014,1287,30,143.481481
132070,2011,1135,79,240.153846
98057,2009,1186,64,187.857143
100590,2009,1237,70,189.000000
49684,2006,1122,114,170.157895
147462,2011,1442,79,295.230769
76367,2007,1442,113,316.850000
18929,2004,1185,52,243.000000
181124,2013,1402,113,285.714286


In [11]:
reg = pd.merge(reg, mean_ranks.rename(columns={'TeamID': 'WTeamID', 
                                                'RankingDayNum':'DayNum', 
                                                'OrdinalRank': 'WRank'}), 
                on=['Season', 'WTeamID', 'DayNum'], how='left')

reg = pd.merge(reg, mean_ranks.rename(columns={'TeamID': 'LTeamID', 
                                                'RankingDayNum':'DayNum', 
                                                'OrdinalRank': 'LRank'}), 
                on=['Season', 'LTeamID', 'DayNum'], how='left')

reg = reg.fillna(1000)

reg.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,WFGM3,WFGA3,WFTM,WFTA,WOR,WDR,WAst,WTO,WStl,WBlk,WPF,LFGM,LFGA,LFGM3,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF,WRank,LRank
0,2003,10,1104,68,1328,62,N,0,27,58,3,14,11,18,14,24,13,23,7,1,22,22,53,2,10,16,22,10,22,8,18,9,2,20,1000.0,1000.0
1,2003,10,1272,70,1393,63,N,0,26,62,8,20,10,19,15,28,16,13,4,4,18,24,67,6,24,9,20,20,25,7,12,8,6,16,1000.0,1000.0
2,2003,11,1266,73,1437,61,N,0,24,58,8,18,17,29,17,26,15,10,5,2,25,22,73,3,26,14,23,31,22,9,12,2,5,23,1000.0,1000.0
3,2003,11,1296,56,1457,50,N,0,18,38,3,9,17,31,6,19,11,12,14,2,18,18,49,6,22,8,15,17,20,9,19,4,3,23,1000.0,1000.0
4,2003,11,1400,77,1208,71,N,0,30,61,6,14,11,13,17,22,12,14,4,4,20,24,62,6,16,17,27,21,15,12,10,7,1,14,1000.0,1000.0


In [12]:
reg['top_team_win'] = 0
reg.loc[reg.LRank <= 30, 'top_team_win'] = 1

reg.top_team_win.mean()

0.010492071699413996

In [13]:
reg['upset'] = 0
reg.loc[reg.WRank - reg.LRank > 15, 'upset'] = 1

reg.upset.mean()

0.053602206135815234

In [14]:
reg['WAway'] = 0
reg.loc[reg.WLoc!='H', 'WAway'] = 1

reg.WAway.mean()

0.4071764046880386

In [15]:
ot_wins = reg[['Season', 'WTeamID', 'NumOT']].copy()

ot_wins['has_OT'] = np.where(ot_wins.NumOT > 0, 1, 0)
ot_wins['OT_win'] = np.where(ot_wins.has_OT > 0, 1, 0)

ot_wins.head()

,Season,WTeamID,NumOT,has_OT,OT_win
0,2003,1104,0,0,0
1,2003,1272,0,0,0
2,2003,1266,0,0,0
3,2003,1296,0,0,0
4,2003,1400,0,0,0


In [16]:
ot_wins = ot_wins.groupby(['Season', 'WTeamID', 'has_OT'], as_index=False).OT_win.mean()

ot_wins.sample(10)

,Season,WTeamID,has_OT,OT_win
1387,2005,1388,1,1
9570,2020,1425,0,0
9342,2020,1272,1,1
9289,2020,1241,0,0
2788,2008,1252,0,0
4696,2011,1423,0,0
8548,2018,1364,0,0
9133,2020,1142,1,1
9421,2020,1325,0,0
6633,2015,1235,0,0


In [17]:
ot_wins[ot_wins.has_OT==1].describe()

,Season,WTeamID,has_OT,OT_win
count,3457.000000,3457.000000,3457.0,3457.0
mean,2011.436216,1285.100376,1.0,1.0
std,4.948174,104.592239,0.0,0.0
min,2003.000000,1101.000000,1.0,1.0
25%,2007.000000,1196.000000,1.0,1.0
50%,2012.000000,1282.000000,1.0,1.0
75%,2016.000000,1377.000000,1.0,1.0
max,2020.000000,1467.000000,1.0,1.0
